In [ ]:
!pip install -q transformers ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.6 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━ 1.1/1.6 MB 36.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 MB 24.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00


In [ ]:

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files


tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("gpt2")


device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
print("Model loaded. Using device:", "GPU" if device == 0 else "CPU")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded. Using device: GPU


In [ ]:

!pip install transformers ipywidgets --quiet

import ipywidgets as widgets
from IPython.display import display, HTML
from transformers import pipeline
from google.colab import files

print("Loading GPT-2 model...")
generator = pipeline("text-generation", model="gpt2", device=-1)
print("✅ Model ready!")

prompt_area = widgets.Textarea(
    value="",
    placeholder="Type a short prompt... e.g. A futuristic city run by robots",
    description="Prompt:",
    layout=widgets.Layout(width='75%', height='120px')
)

style_dd = widgets.Dropdown(
    options=["Normal", "Funny", "Poetic", "Sci-fi", "Romantic"],
    value="Normal",
    description="Style:"
)

sample_list = [
    "A futuristic city run by robots",
    "An alien studying human behaviour",
    "An AI storyteller exploring emotions"
]
sample_dd = widgets.Dropdown(options=sample_list, description="Samples:")

insert_sample_btn = widgets.Button(description="Insert sample", button_style='primary')
generate_btn = widgets.Button(description="Generate", button_style='success')
save_btn = widgets.Button(description="Save as .txt", button_style='info')
download_btn = widgets.Button(description="Download", button_style='warning')


maxlen_slider = widgets.IntSlider(value=100, min=20, max=400, step=10, description='Max length:')
temp_slider = widgets.FloatSlider(value=0.8, min=0.1, max=1.5, step=0.1, description='Temp:')
top_p_slider = widgets.FloatSlider(value=0.9, min=0.1, max=1.0, step=0.05, description='Top-p:')

output_area = widgets.Output()

def insert_sample(_):
    prompt_area.value = sample_dd.value

def run_generation(_):
    with output_area:
        output_area.clear_output()
        raw = prompt_area.value.strip()
        if not raw:
            display(HTML("<p style='color:red'><b>Please enter a prompt first.</b></p>"))
            return

        style = style_dd.value
        if style != "Normal":
            full_prompt = f"Write a {style.lower()} version of the following: {raw}"
        else:
            full_prompt = raw

        max_len = maxlen_slider.value
        temperature = temp_slider.value
        top_p = top_p_slider.value

        generated = generator(
            full_prompt,
            max_length=max_len,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            num_return_sequences=1
        )[0]["generated_text"]

        output_area.generated_result = generated
        display(HTML(f"<h4>Generated ({style})</h4><pre style='white-space:pre-wrap'>{generated}</pre>"))

def save_output(_):
    if hasattr(output_area, 'generated_result'):
        with open("generated_text.txt", "w", encoding="utf-8") as f:
            f.write(output_area.generated_result)
        with output_area:
            display(HTML("<p><b>✅ Saved as 'generated_text.txt' in Colab session.</b></p>"))
    else:
        with output_area:
            display(HTML("<p style='color:red'><b>No generated text. Please generate first.</b></p>"))

def download_output(_):
    if hasattr(output_area, 'generated_result'):
        with open("generated_text.txt", "w", encoding="utf-8") as f:
            f.write(output_area.generated_result)
        files.download("generated_text.txt")
    else:
        with output_area:
            display(HTML("<p style='color:red'><b>No generated text to download.</b></p>"))

insert_sample_btn.on_click(insert_sample)
generate_btn.on_click(run_generation)
save_btn.on_click(save_output)
download_btn.on_click(download_output)

controls_top = widgets.HBox([style_dd, generate_btn, save_btn, download_btn])
controls_bottom = widgets.HBox([maxlen_slider, temp_slider, top_p_slider])

display(HTML("<h2>✨ Text Generation with GPT-2</h2>"))
display(prompt_area)
display(widgets.HBox([sample_dd, insert_sample_btn]))
display(controls_top)
display(controls_bottom)
display(output_area)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.6 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 1.3/1.6 MB 34.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00
Loading GPT-2 model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


✅ Model ready!


Textarea(value='', description='Prompt:', layout=Layout(height='120px', width='75%'), placeholder='Type a shor…

Output()